 ### We will collect data using the Twitter API

In [2]:
import numpy as np
import pandas as pd
import tweepy
import json

In [4]:
consumer_key = "yjzLb8ubK2Ep0IMj1ZE82t1wx"
consumer_secret = "UDzL6BOe5s8O4LOqmE7h8KQFkOMi5E44IsOECQliAazH7mSndb"
access_token = "1322800640863608833-K7VZNTjLVwXpOEXuQuiMnIRGhMKCkf"
access_token_secret = "9RbBryX9wYKMnbUgQSBXFCxZPy3G4zgyClqid19K7mqcf"

In [6]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [7]:
query = "COVID-19 cases in India"

In [12]:
Tweets = api.search(query, count=10, lang='en', exclude="retweets", tweet_mode='extended')
# tweet_mode = 'compat' - Default : only the tweets with <= 140 characters
# tweet_mode = 'extended' - Includes tweets with > 140 chars

# api.search is used for less than 100 tweets, 
# For more than 100 tweets, tweepy.cursor() is used

for tweet in Tweets:
    print(tweet, '\n')
    print('--------------------------------------\n')

Status(_api=<tweepy.api.API object at 0x000001AD82B86E48>, _json={'created_at': 'Sun Nov 01 08:41:03 +0000 2020', 'id': 1322820939311157249, 'id_str': '1322820939311157249', 'full_text': 'IPL 2020: Bowlers’ day out as MI, SRH clinch dominating wins over DC, RCB – Firstcricket News ,\xa0Firstpost https://t.co/iQ07t8ju7X', 'truncated': False, 'display_text_range': [0, 129], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/iQ07t8ju7X', 'expanded_url': 'https://icj24.com/ipl-2020-bowlers-day-out-as-mi-srh-clinch-dominating-wins-over-dc-rcb-firstcricket-news-firstpost/', 'display_url': 'icj24.com/ipl-2020-bowle…', 'indices': [106, 129]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://publicize.wp.com/" rel="nofollow">WordPress.com</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'u

##### The Format is not very user friedly, so, we'll get some attributes

In [10]:
df = pd.DataFrame(columns=["Tweets", "User", "User_Statuses_Count","User_Followers", "User_Location",
                           "User_Verified", "Fav_Count", "RT_Count", "Tweet_Date"])

print(df)
print(df.shape)

Empty DataFrame
Columns: [Tweets, User, User_Statuses_Count, User_Followers, User_Location, User_Verified, Fav_Count, RT_Count, Tweet_Date]
Index: []
(0, 9)


In [19]:
def stream(data, file_name):
    i=0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_Statuses_Count'] = tweet.user.statuses_count
        df.loc[i, 'User_Followers'] = tweet.user.followers_count
        df.loc[i, 'User_Location'] = tweet.user.location
        df.loc[i, 'User_Verified'] = tweet.user.verified
        df.loc[i, 'Fav_Count'] = tweet.favorite_count
        df.loc[i, 'RT_Count'] = tweet.retweet_count
        df.loc[i, 'Tweet_Date'] = tweet.created_at
        
        df.to_excel('{}.xlsx'.format(file_name))
        i+=1
        if(i==1000):
            break

In [21]:
stream(data=[query], file_name="Covid-Cases_Tweets")

In [23]:
df.head()

,Tweets,User,User_Statuses_Count,User_Followers,User_Location,User_Verified,Fav_Count,RT_Count,Tweet_Date
0,RT @timesofindia: India sustains trend of decl...,Farha Naaz Ali,223,17,Earth,False,0,3,2020-11-01 09:50:29
1,RT @COVIDNewsByMIB: #CoronaVirusUpdates: \n\n📍...,NOTTO,15399,650,India,False,0,29,2020-11-01 09:40:53
2,India continues to have one of the lowest COVI...,Millennium Post,96305,822,"New Delhi, India",False,1,0,2020-11-01 09:39:06
3,RT @timesofindia: India sustains trend of decl...,Chinmay Gadre,6099,59,,False,0,3,2020-11-01 09:37:37
4,"18 #Trump rallies resulted in 30,000 #COVID19 ...",Business Today,125464,1053711,India,True,7,0,2020-11-01 09:36:27


### Textblob:

#### Sentiment(polarity, subjectivity)
##### polarity score: [-1, +1]
##### Subjectivity: [0.0, 1.0]
###### Where 0.0 is very Objective and 1.0 is very Subjective

In [24]:
from textblob import TextBlob

### We need to clean the text(tweet), to perform NLP on it

In [84]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [86]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if(analysis.sentiment.polarity > 0):
        return 'Positive'
    elif(analysis.sentiment.polarity == 0):
        return 'Neutral'
    else:
        return 'Negative'

In [87]:
df['Clean_Tweets'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [88]:
df.head()

,Tweets,User,User_Statuses_Count,User_Followers,User_Location,User_Verified,Fav_Count,RT_Count,Tweet_Date,Clean_Tweets,Sentiment
0,RT @timesofindia: India sustains trend of decl...,Farha Naaz Ali,223,17,Earth,False,0,3,2020-11-01 09:50:29,RT India sustains trend of declining Covid 19 ...,Negative
1,RT @COVIDNewsByMIB: #CoronaVirusUpdates: \n\n📍...,NOTTO,15399,650,India,False,0,29,2020-11-01 09:40:53,RT CoronaVirusUpdates Total COVID19 Cases in I...,Neutral
2,India continues to have one of the lowest COVI...,Millennium Post,96305,822,"New Delhi, India",False,1,0,2020-11-01 09:39:06,India continues to have one of the lowest COVI...,Neutral
3,RT @timesofindia: India sustains trend of decl...,Chinmay Gadre,6099,59,,False,0,3,2020-11-01 09:37:37,RT India sustains trend of declining Covid 19 ...,Negative
4,"18 #Trump rallies resulted in 30,000 #COVID19 ...",Business Today,125464,1053711,India,True,7,0,2020-11-01 09:36:27,18 Trump rallies resulted in 30 000 COVID19 ca...,Neutral


In [89]:
df['Sentiment'] = df['Clean_Tweets'].apply(lambda x : analyze_sentiment(x))

In [90]:
df.head()

,Tweets,User,User_Statuses_Count,User_Followers,User_Location,User_Verified,Fav_Count,RT_Count,Tweet_Date,Clean_Tweets,Sentiment
0,RT @timesofindia: India sustains trend of decl...,Farha Naaz Ali,223,17,Earth,False,0,3,2020-11-01 09:50:29,RT India sustains trend of declining Covid 19 ...,Negative
1,RT @COVIDNewsByMIB: #CoronaVirusUpdates: \n\n📍...,NOTTO,15399,650,India,False,0,29,2020-11-01 09:40:53,RT CoronaVirusUpdates Total COVID19 Cases in I...,Neutral
2,India continues to have one of the lowest COVI...,Millennium Post,96305,822,"New Delhi, India",False,1,0,2020-11-01 09:39:06,India continues to have one of the lowest COVI...,Neutral
3,RT @timesofindia: India sustains trend of decl...,Chinmay Gadre,6099,59,,False,0,3,2020-11-01 09:37:37,RT India sustains trend of declining Covid 19 ...,Negative
4,"18 #Trump rallies resulted in 30,000 #COVID19 ...",Business Today,125464,1053711,India,True,7,0,2020-11-01 09:36:27,18 Trump rallies resulted in 30 000 COVID19 ca...,Neutral


## Checking the tweets and sentiments

In [91]:
from random import randint

In [101]:
n = randint(0, 1000)
print("Tweet number", n, end='\n\n')

print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['Clean_Tweets'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Tweet number 706

Original tweet:
 #COVID19 India Tracker | October 31, 2020 (6:30 pm, IST)

➡️Confirmed cases: 8,137,119
➡️Recovered: 7,432,829 (91.3… https://t.co/80ow32PctL

Clean tweet:
 COVID19 India Tracker October 31 2020 6 30 pm IST Confirmed cases 8 137 119 Recovered 7 432 829 91 3

Sentiment of the tweet:
 Positive


In [102]:
df[df.Sentiment=='Positive'].shape[0]

800

In [103]:
df[df.Sentiment=='Negative'].shape[0]

77

In [104]:
df[df.Sentiment=='Neutral'].shape[0]

123